<a href="https://colab.research.google.com/github/thiphan94/EEG-Epileptic-Spike-Detection/blob/main/ProjetDS_M1VMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
sc = spark.sparkContext

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!wget -q https://drive.google.com/drive/folders/1HkqKYNpxZb8HGdct-seQHxjBHx-_Zoux -o exp1-train-400pos.bin


In [ ]:
# Mount data from drive
from google.colab import drive
drive.mount('/content/drive')

#pandas.read_csv("/content/drive/My Drive/DS/exp1-train-400pos.bin")

Mounted at /content/drive


In [ ]:
positive = np.fromfile('/content/drive/My Drive/DS/exp1-train-400pos.bin', dtype=np.float32).reshape([400, 5, 768])
print(positive[0]) # serie a index 0


[[108.586235  114.79495   123.19051   ...  60.54001    58.97502
   57.160065 ]
 [ 74.36134    65.23321    56.587635  ... 174.56192   201.22633
  226.48232  ]
 [ 27.66805    26.607937   25.71433   ...  11.367872    5.6884537
   -1.249531 ]
 [-38.680317  -36.550137  -34.62852   ... -33.736874  -34.02954
  -32.900883 ]
 [-67.347176  -67.13902   -66.79243   ... -14.910572  -14.21761
  -13.904621 ]]


In [ ]:
negative = np.fromfile('/content/drive/My Drive/DS/exp1-train-2000neg.bin', dtype=np.float32).reshape([2000, 5, 768])
print(negative[0]) # serie a index 0

[[   6.789578     3.9754174    2.123387  ...    7.663398    19.70175
    29.822718 ]
 [  52.455315    45.362495    40.24864   ... -126.393745  -135.78629
  -140.47087  ]
 [  22.910574    25.087786    26.635937  ...   33.466297    29.417503
    24.683107 ]
 [ -25.243269   -23.278872   -21.599247  ...   47.55119     38.669083
    29.768295 ]
 [ -15.771972   -17.735523   -19.532282  ...    1.2826554    3.9923859
     6.565947 ]]


In [ ]:
test = np.fromfile('/content/drive/My Drive/DS/exp12-test-1200.bin', dtype=np.float32).reshape([1200, 5, 768])
print(test[0]) # serie a index 0
print(test.shape)

[[ -84.48852   -76.999725  -70.13329  ...   92.868576   89.97075
    86.97282 ]
 [ -54.551765  -52.85429   -50.94947  ... -138.15437  -135.48616
  -130.9657  ]
 [  22.767483   25.895237   28.2063   ...   11.471104   10.629795
     9.655205]
 [  52.576946   59.047306   65.31609  ...   20.003866   18.316921
    16.435457]
 [  -7.797365  -15.986874  -25.377964 ...  -10.434038   -8.745315
    -7.559099]]
(1200, 5, 768)


In [ ]:
# before attache two file positive and negative
print(positive.shape)
print(negative.shape)
# attache two files
new_dataset = np.append(positive,negative, axis=0)
print(new_dataset.shape)
print(new_dataset[0])

(400, 5, 768)
(2000, 5, 768)
(2400, 5, 768)
[[108.586235  114.79495   123.19051   ...  60.54001    58.97502
   57.160065 ]
 [ 74.36134    65.23321    56.587635  ... 174.56192   201.22633
  226.48232  ]
 [ 27.66805    26.607937   25.71433   ...  11.367872    5.6884537
   -1.249531 ]
 [-38.680317  -36.550137  -34.62852   ... -33.736874  -34.02954
  -32.900883 ]
 [-67.347176  -67.13902   -66.79243   ... -14.910572  -14.21761
  -13.904621 ]]


In [ ]:
new_datasetshuffled = np.array(new_dataset)
#shuffle new dataset 
np.random.shuffle(new_datasetshuffled) 
print(new_datasetshuffled.shape)
print(new_datasetshuffled[0])#tester

(2400, 5, 768)
[[ 57.342945   53.550804   47.126568  ... -64.58229   -72.03552
  -77.59227  ]
 [ 41.188187   28.741516   19.04529   ... 138.63673   148.37875
  155.8531   ]
 [-14.799361  -11.327429   -7.9313974 ...  35.093204   37.70457
   39.51087  ]
 [-19.316193  -21.13333   -22.699127  ...  -8.161466   -5.233734
   -2.1882012]
 [ 23.05231    24.917385   27.364973  ...   9.079365   11.223859
   13.255207 ]]


In [ ]:
#index before shuffle
#[index for index, value in enumerate(new_dataset)]

In [ ]:
import random
x = list(enumerate(new_dataset))
random.shuffle(x)
indices, new_dataset = zip(*x)
indices = np.asarray(indices)
print(indices.shape)
label = []

for index, elem in enumerate(indices):
  if 0 <= elem and elem <= 399:
    label.append(1)
  else:
    label.append(0)



print(indices[1:10])
print(label[1:10])

(2400,)
[1410 2124  226 1312 1701 1288  555 1472 2276]
[0, 0, 1, 0, 0, 0, 0, 0, 0]


In [ ]:
from sklearn.decomposition import PCA
data_2d = np.array([features_2d.flatten() for features_2d in new_datasetshuffled])
pca = PCA(n_components=50)
pca.fit(data_2d)
data_pca = pca.transform(data_2d)
print("original shape:   ", data_2d.shape) 
print("transformed shape:", data_pca.shape)

original shape:    (2400, 3840)
transformed shape: (2400, 50)


In [ ]:
from sklearn.decomposition import PCA
testdata_2d = np.array([features_2d.flatten() for features_2d in test])
pca_test = PCA(n_components=50)
pca_test.fit(testdata_2d)
testdata_pca = pca_test.transform(testdata_2d)
print("original shape:   ", testdata_2d.shape)
print("transformed shape:", testdata_pca.shape)

original shape:    (1200, 3840)
transformed shape: (1200, 50)


# New Section